NLP Processing

In [129]:
!pip install --upgrade pip
!pip install re
!pip install typer
!pip install spacy
!pip install scispacy

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [133]:
import pandas as pd
import spacy

In [2]:
import pandas as pd

# Load your file
disease_chems = pd.read_csv('/home/giannis-charalampopoulos/M2PHDS/web based data/wbd-project/CTD_disease_chems_curated.csv')

# Split the PubMedIDs column into lists
disease_chems['PubMedIDs'] = disease_chems['PubMedIDs'].astype(str).str.split('|')

# Explode into multiple rows
disease_chems = disease_chems.explode('PubMedIDs')

# Reset index (optional)
disease_chems = disease_chems.reset_index(drop=True)

# View result
disease_chems.head()

,# Input,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DiseaseCategories,OmimIDs,PubMedIDs,DirectEvidence
0,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,10888601,marker/mechanism
1,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,10958654,marker/mechanism
2,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,17485188,marker/mechanism
3,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,18368630,marker/mechanism
4,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,23055421,marker/mechanism


In [3]:
disease_chems

,# Input,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DiseaseCategories,OmimIDs,PubMedIDs,DirectEvidence
0,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,10888601,marker/mechanism
1,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,10958654,marker/mechanism
2,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,17485188,marker/mechanism
3,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,18368630,marker/mechanism
4,mesh:d002659,succinyladenosine,C043565,4542-23-8,Adenylosuccinate lyase deficiency,MESH:C538235,Genetic disease (inborn)|Mental disorder|Metab...,NaN,23055421,marker/mechanism
...,...,...,...,...,...,...,...,...,...,...
875,mesh:d001321,Vitamin B 12,D014805,68-19-9,Autistic Disorder,MESH:D001321,Mental disorder,NaN,24676906,marker/mechanism
876,mesh:d001321,Vitamin B 6,D025101,8059-24-3,Autistic Disorder,MESH:D001321,Mental disorder,NaN,16494569,marker/mechanism|therapeutic
877,mesh:d001321,Vitamin B 6,D025101,8059-24-3,Autistic Disorder,MESH:D001321,Mental disorder,NaN,16846101,marker/mechanism|therapeutic
878,mesh:d001321,Zinc,D015032,7440-66-6,Autistic Disorder,MESH:D001321,Mental disorder,NaN,28569757,marker/mechanism


In [4]:
pmid_list=disease_chems['PubMedIDs'].astype(str).tolist()

In [6]:
import requests
import time
import pandas as pd
from xml.etree import ElementTree as ET

# Your list of PMIDs
pmid_list = list(set(pmid_list))  # remove duplicates
total = len(pmid_list)

def fetch_pubmed_xml(pmid):
    """Fetch a single PubMed record via NCBI EFetch."""
    url = (
        "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
        f"?db=pubmed&id={pmid}&retmode=xml"
    )

    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to fetch {pmid}")
        return None

    return response.text

records = []
for idx, pmid in enumerate(pmid_list, start=1):

    xml_data = fetch_pubmed_xml(pmid)
    if xml_data is None:
        print(f"❌ Could not retrieve PMID {pmid} ({idx}/{total})")
        continue

    # Parse XML
    root = ET.fromstring(xml_data)
    title = root.findtext(".//ArticleTitle")
    abstract = " ".join(
        [t.text for t in root.findall(".//AbstractText") if t.text]
    )

    records.append({
        "pmid": pmid,
        "title": title,
        "abstract": abstract,
        "xml_raw": xml_data
    })

    # Print progress for every PMID found
    print(f"✔️  Retrieved {idx}/{total} → PMID {pmid}")

    # Optional milestone print every 25 items
    if idx % 25 == 0:
        print(f"⏳ Progress: {idx}/{total} ({100*idx/total:.1f}%)")

    # polite delay (NCBI recommends <10 req/sec)
    time.sleep(0.15)

print("DONE! All available PMIDs retrieved.")
df = pd.DataFrame(records)
print("Total records downloaded:", len(df))

df.head()


✔️  Retrieved 1/208 → PMID 20562626
✔️  Retrieved 2/208 → PMID 3709993
✔️  Retrieved 3/208 → PMID 35254616
✔️  Retrieved 4/208 → PMID 30816183
✔️  Retrieved 5/208 → PMID 36113784
✔️  Retrieved 6/208 → PMID 17690064
✔️  Retrieved 7/208 → PMID 30404997
✔️  Retrieved 8/208 → PMID 25290267
✔️  Retrieved 9/208 → PMID 25641946
✔️  Retrieved 10/208 → PMID 18582451
✔️  Retrieved 11/208 → PMID 19997080
✔️  Retrieved 12/208 → PMID 15782081
✔️  Retrieved 13/208 → PMID 20468076
✔️  Retrieved 14/208 → PMID 25423485
✔️  Retrieved 15/208 → PMID 19154706
✔️  Retrieved 16/208 → PMID 29510222
✔️  Retrieved 17/208 → PMID 32479765
✔️  Retrieved 18/208 → PMID 15238991
✔️  Retrieved 19/208 → PMID 15491979
✔️  Retrieved 20/208 → PMID 15319018
✔️  Retrieved 21/208 → PMID 32353786
✔️  Retrieved 22/208 → PMID 9100317
✔️  Retrieved 23/208 → PMID 16601648
✔️  Retrieved 24/208 → PMID 20013426
✔️  Retrieved 25/208 → PMID 18062943
⏳ Progress: 25/208 (12.0%)
✔️  Retrieved 26/208 → PMID 18368630
✔️  Retrieved 27/208 →

,pmid,title,abstract,xml_raw
0,20562626,Perinatal exposure to hazardous air pollutants...,Hazardous air pollutants are plausible candida...,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."
1,3709993,Effects of fenfluramine on eight outpatients w...,Fenfluramine was administered to eight autisti...,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."
2,35254616,Autism spectrum disorder-like behavior induced...,"Autism spectrum disorders (ASD), also known as...","<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."
3,30816183,Sex Differences in the Effects of Prenatal Bis...,Autism spectrum disorder (ASD) is a neurodevel...,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."
4,36113784,"Prenatal, but not postnatal exposure to chlorp...",The balance between excitatory and inhibitory ...,"<?xml version=""1.0"" ?>\n<!DOCTYPE PubmedArticl..."


In [7]:
pubmed_dataset=df

In [8]:
pubmed_dataset=pubmed_dataset[['pmid', 'abstract']]
pubmed_dataset



,pmid,abstract
0,20562626,Hazardous air pollutants are plausible candida...
1,3709993,Fenfluramine was administered to eight autisti...
2,35254616,"Autism spectrum disorders (ASD), also known as..."
3,30816183,Autism spectrum disorder (ASD) is a neurodevel...
4,36113784,The balance between excitatory and inhibitory ...
...,...,...
203,15205966,There is great evidence in recent years that o...
204,32526256,The manifestations of autism spectrum disorder...
205,16417856,Gestational and genetic factors can contribute...
206,23613074,Valproate is used for the treatment of epileps...


In [9]:
pubmed_dataset = pubmed_dataset[
    pubmed_dataset["abstract"].astype(str).str.strip() != ""
]


In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag

# Downloads
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download("wordnet")

lemmatizer = WordNetLemmatizer()

# Function to convert nltk pos tags to wordnet tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Tokenize + POS tag + Lemmatize
pubmed_dataset["tokens"] = (
    pubmed_dataset["abstract"]
        .astype(str)
        .str.lower()
        .apply(word_tokenize)
        .apply(lambda words: pos_tag(words))
        .apply(lambda tagged: [
            lemmatizer.lemmatize(w, get_wordnet_pos(tag))
            for w, tag in tagged
        ])
)


[nltk_data] Downloading package punkt to /home/giannis-
[nltk_data]     charalampopoulos/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/giannis-charalampopoulos/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/giannis-
[nltk_data]     charalampopoulos/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/tmp/ipykernel_302434/988891669.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubmed_dataset["tokens"] = (


In [11]:
import re

def custom_tokenize(text):
    # keep numbers like 15,000 or 3.5e10 as one token
    pattern = r'\d[\d,\.]*|\w+'
    return re.findall(pattern, text.lower())


In [13]:
pubmed_dataset["tokens"] = pubmed_dataset["tokens"].apply(
    lambda lst: [t for t in lst if any(c.isalnum() for c in t)]
)

/tmp/ipykernel_302434/3246752615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pubmed_dataset["tokens"] = pubmed_dataset["tokens"].apply(


In [14]:
pubmed_dataset=pubmed_dataset.rename(columns={"tokens":"Text"})

In [15]:
pubmed_dataset_processed=pubmed_dataset[["pmid", "Text"]]


In [16]:
merged = pd.merge(
    pubmed_dataset_processed,
    disease_chems,
    left_on="pmid",
    right_on="PubMedIDs",
    how="inner"
)


In [23]:
import pandas as pd
import math
from collections import Counter

# ===================== DATA =====================
final_dataset = merged  # your dataframe

# ===================== RELATION PATTERNS =====================

RELATION_PATTERNS = [
    # Strong causal
    ["cause"], ["causes"], ["caused"],
    ["induce"], ["induces"], ["induced"],
    ["lead", "to"], ["leads", "to"],
    ["trigger"], ["triggers"],

    # Risk-related
    ["increase", "risk"], ["increases", "risk"],
    ["risk", "of"],

    # Associative (explicit)
    ["associated"], ["associated", "with"],
    ["linked"], ["linked", "to"], ["linked", "with"],
    ["related"], ["related", "to"],
    ["correlated", "with"],

    # Biomedical implicit language
    ["involved", "in"],
    ["implicated", "in"],
    ["role", "in"],
    ["plays", "a", "role", "in"],
    ["affects"], ["affect"],
    ["altered"], ["reduced"], ["increased"]
]

# ===================== WEIGHT FUNCTION =====================

def get_weight(pattern):
    base = 1.0 + (0.4 * (len(pattern) - 1))
    text = " ".join(pattern)

    if any(w in text for w in ["cause", "induce", "lead", "trigger"]):
        return base + 2.0
    if "risk" in text:
        return base + 1.5
    if any(w in text for w in ["associated", "linked", "related", "correlated"]):
        return base + 1.0
    return base + 0.5  # biomedical implicit relations

PATTERN_WEIGHTS = {tuple(p): get_weight(p) for p in RELATION_PATTERNS}

# ===================== EXACT PATTERN MATCHING =====================

def match_exact_patterns(tokens):
    score = 0.0
    for pattern, weight in PATTERN_WEIGHTS.items():
        n = len(pattern)
        for i in range(len(tokens) - n + 1):
            if tuple(tokens[i:i+n]) == pattern:
                score += weight
    return score

# ===================== WINDOW-BASED MATCHING (±10 tokens) =====================

RELATION_KEYWORDS = {
    "cause", "causes", "induce", "induces", "risk",
    "associated", "linked", "related", "involved",
    "implicated", "affects", "altered", "reduced", "increased"
}

def match_window_patterns(tokens, window=10):
    score = 0.0
    for i, token in enumerate(tokens):
        if token in RELATION_KEYWORDS:
            left = max(0, i - window)
            right = min(len(tokens), i + window + 1)
            context = tokens[left:right]

            # Require meaningful biomedical context
            if len(context) > 3:
                score += 0.7
    return score

# ===================== BUILD COUNTS =====================

pair_counts = Counter()
pair_pattern_score = Counter()

for _, row in final_dataset.iterrows():
    chem = row["ChemicalName"]
    dis = row["DiseaseName"]
    tokens = row["Text"]
    pair = (chem, dis)

    pair_counts[pair] += 1

    exact_score = match_exact_patterns(tokens)
    window_score = match_window_patterns(tokens)

    pair_pattern_score[pair] += exact_score + window_score

# ===================== FINAL RESULTS =====================

rows = []
for (chem, dis), count in pair_counts.items():
    pattern_score = pair_pattern_score[(chem, dis)]

    evidence_score = math.log(pattern_score + 1) * math.log(count + 2)

    rows.append({
        "ChemicalName": chem,
        "DiseaseName": dis,
        "CoOccurrences": count,
        "WeightedPatternEvidence": round(pattern_score, 2),
        "EvidenceScore": round(evidence_score, 2)
    })

result = pd.DataFrame(rows).sort_values("EvidenceScore", ascending=False)
print(result)


               ChemicalName                             DiseaseName  \
6             Valproic Acid                       Autistic Disorder   
15              Risperidone                       Autistic Disorder   
3             Valproic Acid                Autism Spectrum Disorder   
28                Serotonin                       Autistic Disorder   
2    Diethylhexyl Phthalate                Autism Spectrum Disorder   
..                      ...                                     ...   
124               Neopterin                       Autistic Disorder   
123              Biopterins                       Autistic Disorder   
122              Topiramate  Child Development Disorders, Pervasive   
114             Mirtazapine                       Autistic Disorder   
181  perfluorooctanoic acid                Autism Spectrum Disorder   

     CoOccurrences  WeightedPatternEvidence  EvidenceScore  
6               81                    384.6          26.31  
15              78       

In [24]:
chemicals_to_check = ["Permethrin", "Glutathione", "Thimerosal"]

filtered_check = result[result["ChemicalName"].isin(chemicals_to_check)]

print(filtered_check)


    ChemicalName               DiseaseName  CoOccurrences  \
45   Glutathione         Autistic Disorder             15   
75    Permethrin  Autism Spectrum Disorder              2   
140   Thimerosal         Autistic Disorder              3   

     WeightedPatternEvidence  EvidenceScore  
45                      16.8           8.16  
75                      21.6           4.32  
140                     11.1           4.01  


In [25]:
filtered_check

,ChemicalName,DiseaseName,CoOccurrences,WeightedPatternEvidence,EvidenceScore
45,Glutathione,Autistic Disorder,15,16.8,8.16
75,Permethrin,Autism Spectrum Disorder,2,21.6,4.32
140,Thimerosal,Autistic Disorder,3,11.1,4.01
